In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import os
from IPython.core.debugger import set_trace


# Read text

In [2]:
def cleantxt(fname):
    rawname = fname+'_raw'
    if fname[-3:]=='txt':
        rawname = fname[:-4]+'_raw.txt'
    
    with open(fname, 'r') as f, open(rawname, 'a') as rawf:
        content = f.readlines()
        for line in f:
            rawf.write(line)
            
    # remove whitespace at the end. keep the tab \t to mark level of question    
    content = [x.strip('\n') for x in content] 
   
    for line in content:
        if line[0]==' ': # transform white spaces into tab which is used to count levels
            line = line.replace('  ','\t') 
        if line.find('？')>0: # deal with chinese characters
            line = line.replace('？','?')
        if line.find('。')>0:
            line = line.replace('。','.')
        with open(fname, 'w') as wf: # rewrite the original text name so nothing weird in the processed textfile.
            wf.write(line+'\n')
    return content,cleanname
        

# Text to quesnet
For paper notes, the first line will be the title of this paper, which should be linked to the paper data structure.




In [3]:
"""def getedge(line):
    # get edge
    if line.find('...')>0 and len(line)==line.find('...')+3: # edge of a group head
        ed = line[line.find('_')+1:line.find('...')]
    else: # edge of single node
        ed = line[line.find('_')+1:max([line.rfind('_ '),line.rfind('_@')])]
    #unify the edge label names
    if ed == 'a' or ed.lower()=='answer':
        ed = 'answer' # rename the txt shorthand
    elif ed == 'specify':
        ed = 'specification'
    # get speaker
    if line.find('@')>0: # speaker other than this paper -- usually me
        speaker = line[line.find('@')+1:line.find(' ')]
    else:
        speaker = ''
    return ed,speaker
        """

"def getedge(line):\n    # get edge\n    if line.find('...')>0 and len(line)==line.find('...')+3: # edge of a group head\n        ed = line[line.find('_')+1:line.find('...')]\n    else: # edge of single node\n        ed = line[line.find('_')+1:max([line.rfind('_ '),line.rfind('_@')])]\n    #unify the edge label names\n    if ed == 'a' or ed.lower()=='answer':\n        ed = 'answer' # rename the txt shorthand\n    elif ed == 'specify':\n        ed = 'specification'\n    # get speaker\n    if line.find('@')>0: # speaker other than this paper -- usually me\n        speaker = line[line.find('@')+1:line.find(' ')]\n    else:\n        speaker = ''\n    return ed,speaker\n        "

In [4]:
def getedge(head):
    # get edge
    if head.find('...')>0 and head.find('_')>0: # edge of a group head
        ed = head[head.find('_')+1:head.find('...')]
    else: # edge of single node
        ed = head[head.find('_')+1:head.rfind('_')]
    #unify the edge label names
    if ed == 'a' or ed.lower()=='answer':
        ed = 'answer' # rename the txt shorthand
    elif ed == 'specify':
        ed = 'specification'
    # get speaker
    if head.find('@')>0: # speaker other than this paper -- usually me
        speaker = head[head.find('@')+1:]
    else:
        speaker = ''
    return ed,speaker
def node2label(node):
    if node.find('"')>0:
        label = node[:node.find('"')]
        content = node[node.find('"'):]
    else:
        label = node
        content=""
    return label,content

In [16]:
def txt2quesnet(outdir,fin='',nodedist={},alledge=[]):
    for key in ['label','type','id','content','speaker']:
        nodedist[key]=[]
    with open(fin,'r') as f:
        content = f.readlines()
    
    # create a paper entity
    thisp = {'id':'Ep%d'%np.random.randint(100000)} #E=entity,p=paper
    #FUTURE: may use other ways to record the paper info
    fname = fin[fin.rfind('/')+1:] # get rid of the file directory 
    print(fname)
    thisp['label']=fname.strip('.txt')
    for ks,s in enumerate(fname):
        if s.isdigit():
            thisp['author'] = fname[:ks]
            break
    thisp['year']=fname[ks:ks+4]
    # if the first line is the paper name
    if content[0]=='#':
        thisp['title']=content[0][3:]
        
    # create multiple question threads from each root
    maxlevel = 10 # maximum 10 levels down the tree
    allngrps=[]
    for line in content[1:]:
        if len(line.strip())<1: # skip empty lines
            continue
        elif line[:2]=='##':#ending sign
            break
        else: # a valid new line
            line = line.rstrip()
            # new...
            if line[0].isalpha(): # a root node doesn't start with tab
                    rootlv=0
                    rootlist = [[] for k in range(maxlevel)]# restart a new root list
                    node = line
                    edge = ''
                    spkr = ''
                    head = ''
            else:
                head = line.split(' ')[0]
                if len(line.split(' '))==1:
                    if line[-3:]=='...': #group head
                        nodegrp = [] # for reading parallel nodes
                        hdedge,hdspkr = getedge(head)                    
                        continue
                    elif line.find('end_')>0:
                        allngrps.append(nodegrp)
                        continue
                    else:
                        print(line)
                        print("can't recognize your grammar...")  
                        continue
                else:
                    node = " ".join(line.split(' ')[1:])
                    if head.find('...')<0: # a normal edge; otherwise a member of edge group, edge is given
                        edge,spkr = getedge(head)
                    else:
                        edge,spkr = hdedge,hdspkr
        # Construct the node dictionary, get nid   
        level = int((len(line) - len(line.strip('\t')))/len('\t')) # an absolute level
        label,content=node2label(node)
        if label not in nodedist['label']: 
            if edge == 'answer':
                nodedist['type'].append('Answer') 
                nid = 'a%d'%np.random.randint(100000)
            else:
                nid = 'q%d'%np.random.randint(100000)
                if level>0:
                    nodedist['type'].append('Question')    
                else:
                    nodedist['type'].append('Root question')
            nodedist['label'].append(label)
            nodedist['id'].append(nid)
            nodedist['content'].append(content)
            #nodedist['reference'].append(reference)
            if len(spkr)==0:
                spkr = thisp['id'] # if not specified, default speaker is this paper
            nodedist['speaker'].append(spkr)
        else: # this label already exists, just adding new edge here
            nid = nodedist['id'][nodedist['label'].index(label)]
        if head.find('...')>0:
            nodegrp.append(nid)       

        
        # Construct the edge dictionary    
        rootlist[level]=nid     
        if level>0:
            if len(rootlist[level-1])==0:
                print(node)
                set_trace()
            edgedict = {'start':rootlist[level-1],'end':nid,'label':edge,'id':'l%d'%np.random.randint(100000)}
            if rootlist[level-1] == nid:
                print(level)
                print(line)
                set_trace()
            alledge.append(edgedict)   

            
    nodedf = pd.DataFrame(nodedist)
    edgedf = pd.DataFrame(alledge) 
    #os.makedirs(filedir)
 
    pickle.dump([nodedf,edgedf,allngrps],open(outdir+thisp['label']+'_qnet.p','bw'))
    print('generated database in:\n'+outdir+thisp['label']+'_qnet.p')
    return thisp # output the paper entity. TODO: other entities mentioned in the nodes should also be added, so that return a full database

In [17]:
fname='../txtnotes/papernotes/Pathak2019_Self-Supervised.txt'
outdir='../database/curiosity/algorithms/'
Epaper=txt2quesnet(outdir,fname) 

Pathak2019_Self-Supervised.txt
generated database in:
../database/curiosity/algorithms/Pathak2019_Self-Supervised_qnet.p


In [18]:
[nodedf,edgedf,allngrps] = pickle.load(open('../database/curiosity/algorithms/Pathak2019_Self-Supervised_qnet.p','rb'))
edgedf

,end,id,label,start
0,a50528,l33834,answer,q98168
1,q11797,l71951,specification,a50528
2,a29489,l77256,answer,q11797
3,q50758,l79058,specification,a50528
4,q61308,l54872,specification,q50758
5,a49989,l89762,answer,q61308
6,q15285,l90043,,a49989
7,q95112,l94937,comment,q15285
8,q29690,l97810,specification,q50758
9,a88791,l43565,answer,q29690


In [19]:
nodedf

,label,type,id,content,speaker
0,What is the algorithm?,Root question,q98168,,Ep4056
1,RL with intrinsic reward derived from world mo...,Answer,a50528,,Ep4056
2,What is the reward?,Question,q11797,,Ep4056
3,Variance across the prediction from all models...,Answer,a29489,,Ep4056
4,What is the training process?,Question,q50758,,Ep4056
5,What is the policy optimization algorithm?,Question,q61308,,Ep4056
6,Using direct gradients to maximize internal re...,Answer,a49989,,Ep4056
7,How to optimize for multi-step reward horizon?,Question,q15285,,Ep4056
8,Could apply the policy optimizer recursively a...,Question,q95112,,Ep4056
9,How to train the world model?,Question,q29690,,Ep4056


next step: use "pickle to networkx" notebook to do the visualization